In [1]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
import pickle

In [ ]:
# We must create a training dataset for the model. 

# We want a dataset that has one row for each acute inpatient encounter that qualifies as an index admission.

# We create that dataset by running the following query against the Tuva Data Model:

```sql
select 
  bb.sex,
  (aa.admit_date - bb.birth_date) as age_at_admit,
  aa.length_of_stay,
  aa.discharge_disposition_code,
  aa.drg_code,
  aa.diagnosis_ccs,
  aa.specialty_cohort,
  aa.unplanned_readmit_30_flag
from tuva.readmissions.readmission_summary aa
left join tuva.core.patient bb
on aa.person_id = bb.person_id
where aa.index_admission_flag = 1
```

In [ ]:
# The grain of the training dataset is one row per acute inpatient encounter.

# We only consider acute inpatient encounters that qualify as index admissions, 
# hence the "where" clause in the query above (where index_admission_flag = 1).

# The first 7 columns in the dataset are the features (the inputs that are used to predict the output):
#     - SEX
#     - AGE_AT_ADMIT
#     - LENGTH_OF_STAY
#     - DISCHARGE_DISPOSITION_CODE
#     - DRG_CODE
#     - DIAGNOSIS_CCS
#     - SPECIALTY_COHORT

# The last column in the dataset is the target (the output we are trying to predict):
#     - unplanned_readmit_30_flag (0 = the encounter did not have an unplanned 30-day readmission, 
#                                  1 = the encounter did have an unplanned 30-day readmission)

In [2]:
# Load the dataset (this is the CSV file that is the output of the query above,
#                   which in this case we call 'readmissions_training_data.csv')
df = pd.read_csv('readmissions_training_data.csv')  # Replace with your full path if needed
df.head()

,SEX,AGE_AT_ADMIT,LENGTH_OF_STAY,DISCHARGE_DISPOSITION_CODE,DRG_CODE,DIAGNOSIS_CCS,SPECIALTY_COHORT,UNPLANNED_READMIT_30_FLAG
0,male,27825,2,1,247,100,Cardiovascular,0
1,male,25033,2,1,445,149,Medicine,0
2,male,25094,4,1,871,2,Medicine,0
3,male,25166,2,1,266,96,Cardiovascular,0
4,female,30797,2,6,183,231,Medicine,0


In [ ]:
# We label encode categorical columns.
#
# The following columns contain discrete values from pre-defined sets of valid values (string values):
#     - SEX
#     - DISCHARGE_DISPOSITION_CODE
#     - DRG_CODE
#     - DIAGNOSIS_CCS
#     - SPECIALTY_COHORT
# 
# We must encode the values for these columns as integers because the machine learning models 
# we will use cannot natively process string values and, instead, expect numeric inputs. 

# The following code maps each distinct string value in these columns to a different integer value.

# These integer values are just used as category labels by the model, 
# not as an ordinal feature (i.e. order does not matter).

# Since we use tree-based models (XGBoost and Random Forest), these models treat the integer values
# as discrete splits, not as ordered magnitudes (i.e. the integers are treated as groupings, not as scales).
# Note that for linear models it is not recommended to encode categorical features in this way because
# those types of models treat the integer encodings as ordered quantities. 
# For linear models, other enconding methods are recommended for categorical features (e.g. one-hot encoding).

label_encoders = {}
categorical_cols = ['SEX','DISCHARGE_DISPOSITION_CODE', 'DRG_CODE', 'DIAGNOSIS_CCS', 'SPECIALTY_COHORT']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
df.head()

In [ ]:
# Normalize numeric features so that numeric features with different
# scales are all normalized to have mean = 0 and standard deviation = 1.
# This may help with model convergence and can prevent features with larger scales
# dominating other features.

scaler = StandardScaler()
df[['AGE_AT_ADMIT', 'LENGTH_OF_STAY']] = scaler.fit_transform(df[['AGE_AT_ADMIT', 'LENGTH_OF_STAY']])
df.head()

In [ ]:
# Train-test split:
# We split the dataset into a training set (80% of the data) with which we'll train the model,
# and a test set (20% of the data) on which we'll test model performance.

X = df.drop(columns=['UNPLANNED_READMIT_30_FLAG'])
y = df['UNPLANNED_READMIT_30_FLAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Train models (Random Forest and XGBoost) on the training set:

rf_model = RandomForestClassifier(random_state=42)
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)

rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

In [ ]:
# Evaluate models

# This function 
def evaluate_model(model, name):
    print(f"\n=== {name} - Training Set ===")
    y_pred_train = model.predict(X_train)
    y_proba_train = model.predict_proba(X_train)[:, 1]
    print(classification_report(y_train, y_pred_train))
    print("ROC AUC:", roc_auc_score(y_train, y_proba_train))

    print(f"\n=== {name} - Test Set ===")
    y_pred_test = model.predict(X_test)
    y_proba_test = model.predict_proba(X_test)[:, 1]
    print(classification_report(y_test, y_pred_test))
    print("ROC AUC:", roc_auc_score(y_test, y_proba_test))

evaluate_model(rf_model, "Random Forest")
evaluate_model(xgb_model, "XGBoost")

In [ ]:
# Feature importance plot
def plot_feature_importance(model, model_name, feature_names):
    importances = model.feature_importances_
    indices = importances.argsort()[::-1]
    plt.figure(figsize=(10, 6))
    plt.title(f"{model_name} Feature Importances")
    plt.bar(range(len(importances)), importances[indices], align='center')
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

plot_feature_importance(rf_model, "Random Forest", X.columns)
plot_feature_importance(xgb_model, "XGBoost", X.columns)

In [ ]:
# Save the model to a file:
with open('readmissions_predictive_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)